<a href="https://colab.research.google.com/github/jalew188/PeptDeep-HLA/blob/master/nbs/HLA1_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of presentation of HLA-I peptides

In [ ]:
%pip install -q git+https://github.com/MannLabs/PeptDeep-HLA.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MannLabs/PeptDeep-HLA.git to /tmp/pip-req-build-2le68jin
  Running command git clone -q https://github.com/MannLabs/PeptDeep-HLA.git /tmp/pip-req-build-2le68jin
     |████████████████████████████████| 552 kB 4.9 MB/s 
     |████████████████████████████████| 1.4 MB 41.9 MB/s 
     |████████████████████████████████| 205 kB 67.0 MB/s 
     |████████████████████████████████| 3.2 MB 28.9 MB/s 
     |████████████████████████████████| 5.8 MB 54.8 MB/s 
     |████████████████████████████████| 9.2 MB 22.0 MB/s 
     |████████████████████████████████| 3.1 MB 66.4 MB/s 
     |████████████████████████████████| 50 kB 2.8 MB/s 
     |████████████████████████████████| 4.7 MB 55.0 MB/s 
     |████████████████████████████████| 182 kB 78.9 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 164 kB 62.1 MB/s 
     |██████████████

In [ ]:
%pip install -q gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import gdown
HLA1_seq_url = "https://drive.google.com/file/d/1MMKbqfT86SXAj37F4MPTJg6NWXDGC-Id/view?usp=share_link"
fasta_url = "https://drive.google.com/file/d/1V9KxDniKwZFZnHlP58EbjkuelNJnp1Kq/view?usp=share_link"
seq_file = r'IEDB_HLA1_seqs.tsv'
save_as = r'HLA1_IEDB.pt'
fasta = r'UP000005640_human_reviewed.fasta'
pretrained_model = None

gdown.download(HLA1_seq_url, seq_file, fuzzy=True)
gdown.download(fasta_url, fasta, fuzzy=True)


Downloading...
From: https://drive.google.com/uc?id=1MMKbqfT86SXAj37F4MPTJg6NWXDGC-Id
To: /content/IEDB_HLA1_seqs.tsv
100%|██████████| 8.45M/8.45M [00:00<00:00, 190MB/s]
Downloading...
From: https://drive.google.com/uc?id=1V9KxDniKwZFZnHlP58EbjkuelNJnp1Kq
To: /content/UP000005640_human_reviewed.fasta
100%|██████████| 13.6M/13.6M [00:00<00:00, 78.5MB/s]


'UP000005640_human_reviewed.fasta'

In [ ]:
### transfer learning
# from peptdeep_hla.HLA_class_I import pretrained_HLA1
# pretrained_model = pretrained_HLA1

For transfer learning of sample-specific models, please set pretrained_model as an existing model file and use sample-specific sequences instead of seq_file.

In [ ]:
import pandas as pd
pep_df = pd.read_csv(seq_file)
pep_df['nAA'] = pep_df.sequence.str.len()
pep_df['HLA'] = 1
pep_df

,sequence,nAA,HLA
0,AAAAAAAAAAAAA,13,1
1,AAAAPYAGW,9,1
2,AAAARAAAL,9,1
3,AAAATCALV,9,1
4,AAAKAAAAV,9,1
...,...,...,...
705738,VVVSIDRFLR,10,1
705739,VYNMVVKL,9,1
705740,WGKSKEWGRNCKGCN,15,1
705741,YASLRSLV,8,1


In [ ]:
def split_to_two_dfs(df, ratio=0.8):
    train_df = df.sample(frac=ratio, replace=False)
    test_df = df.drop(train_df.index)
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

pos_train_df, pos_test_df = split_to_two_dfs(pep_df, 0.8)

In [ ]:
from peptdeep_hla.utils import get_random_sequences

def concat_neg_df(pos_df, prot_df):
    df_list = [pos_df]
    for nAA, group_df in pos_df.groupby('nAA'):
        rnd_seqs = get_random_sequences(
            prot_df, 
            n=len(group_df),
            pep_len = nAA
        )
        df_list.append(pd.DataFrame(
            dict(sequence=rnd_seqs,nAA=nAA,HLA=0)
        ))
    return pd.concat(df_list).reset_index(drop=True)

In [ ]:
from peptdeep_hla.HLA_class_I import HLA_Class_I_Classifier

2022-12-29 01:44:27> Downloading pretrained_models.zip ...
2022-12-29 01:44:28> The pretrained models had been downloaded in /root/peptdeep/pretrained_models/pretrained_models.zip


## Build, train and predict

In [ ]:
model = HLA_Class_I_Classifier(
    fasta_files=[fasta],
)
if pretrained_model:
    model.load(pretrained_model)
model.get_parameter_num()

1669697

In [ ]:
model.train(pos_train_df, epoch=100, warmup_epoch=20, verbose=True)

[Training] Epoch=1, lr=5e-06, loss=0.6933580278674235
[Training] Epoch=2, lr=1e-05, loss=0.6754722594088102
[Training] Epoch=3, lr=1.5e-05, loss=0.5297332382382427
[Training] Epoch=4, lr=2e-05, loss=0.480515140385168
[Training] Epoch=5, lr=2.5e-05, loss=0.45667302816937233
[Training] Epoch=6, lr=3e-05, loss=0.4495536099084158
[Training] Epoch=7, lr=3.5e-05, loss=0.44047606413891727
[Training] Epoch=8, lr=4e-05, loss=0.4389060655703842
[Training] Epoch=9, lr=4.5e-05, loss=0.43643731560508786
[Training] Epoch=10, lr=5e-05, loss=0.4360491626654771
[Training] Epoch=11, lr=5.500000000000001e-05, loss=0.429061990097475
[Training] Epoch=12, lr=6e-05, loss=0.4286614368002446
[Training] Epoch=13, lr=6.500000000000001e-05, loss=0.4262140641703723
[Training] Epoch=14, lr=7e-05, loss=0.42403797937701465
[Training] Epoch=15, lr=7.500000000000001e-05, loss=0.41956930462489284
[Training] Epoch=16, lr=8e-05, loss=0.4148490408079836
[Training] Epoch=17, lr=8.5e-05, loss=0.41393732207709316
[Training] E

In [ ]:
model.save(save_as)

In [ ]:
test_df = concat_neg_df(pos_test_df, model.protein_df)
model.predict(test_df)

,sequence,nAA,HLA,HLA_prob_pred
0,AAAAAAAAAAAAA,13,1,0.753656
1,AAAATCALV,9,1,0.140352
2,AAAKAAAAV,9,1,0.848738
3,AAAKTPVIVV,10,1,0.798607
4,AAAYFVGYLK,10,1,0.598994
...,...,...,...,...
282293,GGRPRRTGLLSVELRVLDENDNPPVFEQDEYRAAVREDAQPGAE,44,0,0.804847
282294,GHQDSDSDLSPPRNRPRHRSSDSDLSPPRRRQRTKSSDSDLSPPR,45,0,0.637659
282295,LPQRGQSNRGFHGFLPEDIKKEAARASRKICFVCKKKGAAINCQKD...,50,0,0.043602
282296,HLKLCAGIMITASHNPKQDNGYKVYWDNGAQIISPHDKGISQAIEE...,51,0,0.365074


In [ ]:
prob=0.7
print("Precision =", test_df[test_df.HLA_prob_pred>prob]['HLA'].mean())
print("Recall =", test_df[test_df.HLA_prob_pred>prob]['HLA'].sum()/len(test_df)*2)
print("False Positive Rate =", 1-(1-test_df[test_df.HLA_prob_pred<prob]['HLA']).sum()/len(test_df)*2)

Precision = 0.8812696811009239
Recall = 0.7177450778964073
False Positive Rate = 0.09669923272570125


## Get all sequences with >0.7 predicted probabilities

In [ ]:
hla_df = model.predict_from_proteins(prob_threshold=prob)
hla_df

/usr/local/lib/python3.8/dist-packages/pydivsufsort/divsufsort.py:76: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")
/usr/local/lib/python3.8/dist-packages/alphabase/protein/lcp_digest.py:7: UserWarning: converting str argument uses more memory
  lcp_array = kasai(cat_prot, suffix_array)
100%|██████████| 72/72 [1:00:02<00:00, 50.03s/it]


,start_pos,end_pos,nAA,HLA_prob_pred,sequence
0,4709940,4709948,8,0.791963,ASVDYIRK
1,4709941,4709949,8,0.864838,SVDYIRKL
2,4709943,4709951,8,0.875017,DYIRKLQR
3,4709951,4709959,8,0.884643,EQQRAKEL
4,4709958,4709966,8,0.834065,LENRQKKL
...,...,...,...,...,...
6025465,2563511,2563525,14,0.947904,SRLDTPLGQTLIRM
6025466,535531,535545,14,0.814335,SADEFGILETDVTF
6025467,4323141,4323155,14,0.712731,MGMMNNPNPYGSPY
6025468,984573,984587,14,0.777980,AKAVYDFKAQTSKE


In [ ]:
# hla_df.to_csv('Predicted_HLA.tsv',index=False, sep="\t")